In [ ]:
#!nvidia-smi

In [ ]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import os
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import random

In [ ]:
mnist = input_data.read_data_sets("./data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

In [ ]:
# hyper parameters
learning_rate = 0.01
training_epochs = 70
batch_size = 50
n_class = 10
seed = 777
num_epochs_per_decay = 10
tf.set_random_seed(seed)
cur_dir = os.getcwd()

In [ ]:
def preproc(x):
    # x = x*2 - 1.0
    # per-example mean subtraction (http://ufldl.stanford.edu/wiki/index.php/Data_Preprocessing)
    mean = tf.reduce_mean(x, axis=1, keep_dims=True)
    return x - mean

In [ ]:
def conv_bn_activ(name, x, n_filters, kernel_size, strides, training, seed, padding='SAME'):
    with tf.variable_scope(name):
        net = tf.layers.conv2d(x, n_filters, kernel_size, strides=strides, padding=padding, use_bias=False,
                              kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        net = tf.layers.batch_normalization(net, training=training)
        net = tf.nn.relu(net)
    return net

In [ ]:
def residual_block(name, x, n_filters, training, seed, downsample=False):
    if downsample:
        strides = 2
    else:
        strides = 1
    with tf.variable_scope(name):
        with tf.variable_scope("inner_conv1"):
            net1 = tf.layers.conv2d(x, n_filters, [3, 3], strides=strides, padding='SAME', use_bias=False,
                                   kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
            net1 = tf.layers.batch_normalization(net1, training=training)
            net1 = tf.nn.relu(net1)
        with tf.variable_scope("inner_conv2"):
            net2 = tf.layers.conv2d(net1, n_filters, [3, 3], strides=1, padding='SAME', use_bias=False,
                                   kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
            net2 = tf.layers.batch_normalization(net2, training=training)
        if downsample:
            x = tf.layers.conv2d(x, n_filters, [1, 1], strides=2, padding='SAME',
                                kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        return tf.nn.relu(net2 + x)

In [ ]:
def build_resnet(X_img, layer_n, training, seed):
    net = X_img
    with tf.variable_scope("conv0"):
        net = conv_bn_activ(name="pre_conv", x=net, n_filters=16, kernel_size=[3,3], strides=1, 
                            training=is_train, seed=seed)
    with tf.variable_scope("conv1"):
        for i in range(layer_n):
            net = residual_block(name="resblk{}".format(i), x=net, n_filters=16, training=training, 
                                 seed=seed)
            print(net)
    with tf.variable_scope("conv2"):
        for i in range(layer_n):
            net = residual_block(name="resblk{}".format(i), x=net, n_filters=32, training=training, 
                                 seed=seed, downsample=(i==0))
            print(net)
    with tf.variable_scope("conv3"):
        for i in range(layer_n):
            net = residual_block(name="resblk{}".format(i), x=net, n_filters=64, training=training, 
                                 seed=seed, downsample=(i==0))
            print(net)
    with tf.variable_scope("fc"):
        net = tf.layers.average_pooling2d(name="gap", inputs=net, pool_size=[7, 7], 
                                          strides=7, padding='SAME')
        print(net)
        net = tf.reshape(net, [-1, 64])
        print(net)
        logits = tf.layers.dense(net, 10, name="logits", 
                              kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
        print(logits)
    return logits

In [ ]:
X = tf.placeholder(tf.float32, [None, 784], name="X")
Y = tf.placeholder(tf.float32, [None, n_class], name = "Y")
is_train = tf.placeholder(tf.bool, name="is_train")

In [ ]:
X_pre = preproc(X)
X_img = tf.reshape(X_pre, [-1, 28, 28, 1], name="X_img")

In [ ]:
logits = build_resnet(X_img, layer_n=3, training=is_train, seed=seed)

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y), name="loss")
n_batches_per_epoch = int(mnist.train.num_examples / batch_size)
decay_steps = int(n_batches_per_epoch * num_epochs_per_decay)
global_step = tf.Variable(0, trainable=False)
learningRate = tf.train.exponential_decay(learning_rate=learning_rate,
                                          global_step= global_step,
                                          decay_steps=decay_steps,
                                          decay_rate= 0.15,
                                          staircase=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(
        loss, global_step=global_step, name="optimizer") 

In [ ]:
pred = tf.argmax(logits, axis=1, name="prediction")
prob = tf.nn.softmax(logits, name="softmax")
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y, axis=1)), tf.float32), name="accuracy")

In [ ]:
train_var = [X, Y, is_train, logits, pred, prob, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])
tf.add_to_collection('train_var', train_var[4])
tf.add_to_collection('train_var', train_var[5])
tf.add_to_collection('train_var', train_var[6])
saver = tf.train.Saver()

In [ ]:
savedir = 'checkpoints'
if not os.path.exists(savedir):
    os.makedirs(savedir)
saver.export_meta_graph(os.path.join(cur_dir, savedir, 'mnist_resnet20.meta'), collection_list=['train_var'])

In [ ]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [ ]:
# train my model
print('Learning started. It takes sometime.')
max_test_acc = 0.
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    
    total_batch = int(mnist.train.num_examples / batch_size)
    total_batch_test = int(mnist.test.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys, is_train:True}
        acc, c, _ = sess.run([accuracy, loss, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    for i in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys, is_train:False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test
    if avg_test_acc > max_test_acc:
        max_test_acc = avg_test_acc
        print('saving a graph and weights => accuracy : {}'.format(max_test_acc))
        saver.save(sess, os.path.join(cur_dir, savedir, 'mnist_resnet20_50.ckpt'))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 
          'train accuracy = ', '{:.5f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.5f}'.format(avg_test_acc))


print('Learning Finished!')

In [ ]:
#saver.save(sess, os.path.join(cur_dir, 'checkpoints', 'mnist_save.ckpt'))

In [ ]:
def evaluate(X_sample, y_sample, batch_size=100):
    """Run a minibatch accuracy op"""

    N = X_sample.shape[0]
    correct_sample = 0

    for i in range(0, N, batch_size):
        X_batch = X_sample[i: i + batch_size]
        y_batch = y_sample[i: i + batch_size]
        N_batch = X_batch.shape[0]

        feed = {
            X: X_batch,
            Y: y_batch,
            is_train: False
        }

        correct_sample += sess.run(accuracy, feed_dict=feed) * N_batch

    return correct_sample / N

print("\nAccuracy Evaluates")
print("-------------------------------")
print('Train Accuracy:', '{:.5f}'.format(evaluate(mnist.train.images, mnist.train.labels)))
print('Test Accuracy:', '{:.5f}'.format(evaluate(mnist.test.images, mnist.test.labels)))